# Notebook : Entraînement d'un Transformer simple pour la prédiction du prochain token

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


Les séquences générées par Agent13

In [2]:
Training_sequences = [
[[0, 1], [1, 1], [1, 3], [3, 1], [1, 4], [4, 1], [1, 7], [7, 1], [1, 8], [8, 0], [0, 2], [2, 0], [3, 3], [3, 4], [4, 3], [3, 7], [7, 3], [3, 8], [3, 10], [10, 1], [1, 10], [10, 2], [5, 1], [3, 5], [5, 3], [3, 6], [6, 1], [1, 5], [5, 5], [5, 6], [6, 3], [5, 8], [8, 1], [7, 4], [4, 4], [4, 7], [7, 7], [7, 8], [0, 3], [6, 5], [5, 10]],
[[0, 1, 1], [0, 1, 1], [1, 1, 3], [1, 1, 3], [1, 3, 1], [1, 3, 1], [3, 1, 4], [3, 1, 4], [1, 4, 1], [1, 4, 1], [4, 1, 7], [4, 1, 7], [1, 7, 1], [1, 7, 1], [7, 1, 8], [7, 1, 8], [1, 8, 0], [1, 8, 0], [8, 0, 2], [8, 0, 2], [0, 2, 0], [0, 2, 0], [2, 0, 3], [3, 3, 4], [3, 3, 4], [3, 4, 3], [3, 4, 3], [4, 3, 7], [4, 3, 7], [3, 7, 3], [3, 7, 3], [7, 3, 8], [7, 3, 8], [3, 10, 1], [3, 10, 1], [10, 1, 10], [10, 1, 10], [1, 10, 2], [1, 10, 2], [5, 1, 3], [5, 1, 3], [1, 3, 3], [1, 3, 3], [3, 3, 5], [3, 3, 5], [3, 5, 3], [3, 5, 3], [5, 3, 6], [5, 3, 6], [3, 6, 1], [3, 6, 1], [6, 1, 5], [6, 1, 5], [1, 5, 5], [1, 5, 5], [5, 5, 6], [5, 5, 6], [5, 6, 3], [5, 6, 3], [6, 3, 3], [6, 3, 3], [3, 5, 8], [3, 5, 8], [5, 8, 1], [5, 8, 1], [8, 1, 7], [8, 1, 7], [1, 7, 4], [1, 7, 4], [7, 4, 4], [7, 4, 4], [4, 4, 7], [4, 4, 7], [4, 7, 7], [4, 7, 7], [7, 7, 8], [7, 7, 8], [8, 0, 3], [6, 5, 10], [6, 5, 10], [2, 0, 3]],
[[0, 1, 1, 3], [1, 1, 3, 1], [1, 3, 1, 4], [3, 1, 4, 1], [1, 4, 1, 7], [4, 1, 7, 1], [1, 7, 1, 8], [7, 1, 8, 0], [1, 8, 0, 2], [8, 0, 2, 0], [2, 0, 2, 0], [2, 0, 2, 0], [0, 2, 0, 3], [0, 2, 0, 3], [2, 0, 3, 3], [2, 0, 3, 3], [3, 3, 4, 3], [3, 4, 3, 7], [4, 3, 7, 3], [3, 7, 3, 8], [3, 8, 0, 2], [3, 8, 0, 2], [3, 10, 1, 10], [10, 1, 10, 2], [2, 0, 3, 5], [5, 1, 3, 3], [1, 3, 3, 5], [3, 3, 5, 3], [3, 5, 3, 6], [5, 3, 6, 1], [3, 6, 1, 5], [6, 1, 5, 5], [1, 5, 5, 6], [5, 5, 6, 3], [5, 6, 3, 3], [6, 3, 3, 5], [3, 3, 5, 8], [3, 5, 8, 1], [5, 8, 1, 7], [8, 1, 7, 4], [1, 7, 4, 4], [7, 4, 4, 7], [4, 4, 7, 7], [4, 7, 7, 8], [0, 2, 0, 3], [7, 8, 0, 3], [10, 2, 0, 3], [0, 2, 0, 2]],
[[0, 2, 0, 2, 0], [2, 0, 2, 0, 3], [0, 2, 0, 3, 3], [2, 0, 3, 3, 4], [7, 3, 8, 0, 2], [0, 2, 0, 3, 10], [0, 2, 0, 3, 10], [10, 2, 0, 2, 0], [10, 2, 0, 2, 0], [2, 0, 3, 5, 1], [2, 0, 3, 5, 1], [8, 0, 2, 0, 3], [7, 8, 0, 3, 6], [5, 10, 2, 0, 3], [5, 10, 2, 0, 3], [8, 0, 2, 0, 2]],
[[8, 0, 2, 0, 2, 0], [8, 0, 2, 0, 2, 0], [0, 2, 0, 2, 0, 3], [0, 2, 0, 2, 0, 3], [0, 2, 0, 3, 10, 1], [1, 10, 2, 0, 2, 0], [0, 2, 0, 2, 0, 3], [2, 0, 3, 5, 1, 3], [7, 8, 0, 2, 0, 3], [7, 8, 0, 3, 6, 5], [7, 8, 0, 3, 6, 5], [6, 5, 10, 2, 0, 3], [7, 8, 0, 2, 0, 2]]
]

In [3]:
class TokenDataset(Dataset):
    def __init__(self, sequences, max_length=10):
        self.sequences = sequences
        self.max_length = max_length
        
    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        input_seq = seq[:-1]
        target_seq = seq[1:]
        input_seq = input_seq + [0] * (self.max_length - len(input_seq))
        target_seq = target_seq + [0] * (self.max_length - len(target_seq))
        return torch.tensor(input_seq), torch.tensor(target_seq)

def create_dataloader(sequences, batch_size=2, max_length=10):
    dataset = TokenDataset(sequences, max_length=max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader


In [4]:

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size=12, d_model=32, nhead=4, num_layers=2, max_length=10):
        super(SimpleTransformer, self).__init__()
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(max_length, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.max_length = max_length

    def forward(self, x):
        batch_size, seq_len = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        x = self.token_emb(x) + self.pos_emb(positions)
        x = x.permute(1, 0, 2)
        out = self.transformer_encoder(x)
        out = out.permute(1, 0, 2)
        logits = self.fc_out(out)
        return logits


In [25]:

def predict_next_token_probs(model, input_sequence, max_length=10, device='cpu'):
    model.eval()
    if len(input_sequence) > max_length:
        raise ValueError(f"Input sequence length {len(input_sequence)} exceeds maximum {max_length}")
    input_seq = input_sequence + [0] * (max_length - len(input_sequence))
    input_tensor = torch.tensor(input_seq, device=device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_tensor)
    last_index = len(input_sequence) - 1
    logits_for_next_token = logits[0, last_index, :]
    # Pairwise probabilities
    pairwise_logits = logits[0, -1, :].reshape(-1, 2)
    probabilities = nn.functional.softmax(pairwise_logits, dim=1).flatten().tolist()
    return probabilities
    # probs = F.softmax(logits_for_next_token, dim=0)
    # return probs


In [26]:

def train_model(model, dataloader, num_epochs=10, learning_rate=1e-3, device='cpu'):
    model.train()
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.view(-1, outputs.size(-1))
            targets = targets.view(-1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Époque [{epoch+1}/{num_epochs}], Perte moyenne : {avg_loss:.4f}")


On instancie le modèle

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleTransformer(vocab_size=12, max_length=10).to(device)

In [36]:
for example_sequences in Training_sequences:
    print(example_sequences)
    dataloader = create_dataloader(example_sequences, batch_size=16, max_length=10)
    train_model(model, dataloader, num_epochs=20, learning_rate=1e-3, device=device)

[[0, 1], [1, 1], [1, 3], [3, 1], [1, 4], [4, 1], [1, 7], [7, 1], [1, 8], [8, 0], [0, 2], [2, 0], [3, 3], [3, 4], [4, 3], [3, 7], [7, 3], [3, 8], [3, 10], [10, 1], [1, 10], [10, 2], [5, 1], [3, 5], [5, 3], [3, 6], [6, 1], [1, 5], [5, 5], [5, 6], [6, 3], [5, 8], [8, 1], [7, 4], [4, 4], [4, 7], [7, 7], [7, 8], [0, 3], [6, 5], [5, 10]]
Époque [1/20], Perte moyenne : 4.5009
Époque [2/20], Perte moyenne : 2.8230
Époque [3/20], Perte moyenne : 2.5124
Époque [4/20], Perte moyenne : 2.3704
Époque [5/20], Perte moyenne : 2.3127
Époque [6/20], Perte moyenne : 2.1738
Époque [7/20], Perte moyenne : 2.0714
Époque [8/20], Perte moyenne : 1.8600
Époque [9/20], Perte moyenne : 1.8573
Époque [10/20], Perte moyenne : 1.8303
Époque [11/20], Perte moyenne : 1.8040
Époque [12/20], Perte moyenne : 1.6494
Époque [13/20], Perte moyenne : 1.8957
Époque [14/20], Perte moyenne : 1.8013
Époque [15/20], Perte moyenne : 1.8641
Époque [16/20], Perte moyenne : 1.8372
Époque [17/20], Perte moyenne : 1.8302
Époque [18/2

In [34]:
example_input = [2]  # Expect next token 0
probs = predict_next_token_probs(model, example_input, max_length=10, device=device)
# print("\nProbabilités pour chaque token (index 0-11) :")
# print(probs.cpu().numpy())


In [35]:
import pandas as pd

expected_df = pd.DataFrame({
    'token':  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    'action': [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    'outcome':[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'valence':[5, -10, -1, -1, -1, -1, -1, -1, -3, -3, -3, -3],
    'probability': probs})
# expected_df['expected_valence'] = expected_df['valence'] * expected_df['probability']
# action_expectation_df = expected_df.groupby('action').agg({'expected_valence': 'sum'}).reset_index()
# action_expectation_df = action_expectation_df.sort_values(by=['expected_valence'], ascending=[False]).reset_index(drop=True)
expected_df

,token,action,outcome,valence,probability
0,0,0,0,5,0.018175
1,1,0,1,-10,0.981825
2,2,1,0,-1,0.039293
3,3,1,1,-1,0.960707
4,4,2,0,-1,0.514183
5,5,2,1,-1,0.485817
6,6,3,0,-1,0.375161
7,7,3,1,-1,0.624839
8,8,4,0,-3,0.964635
9,9,4,1,-3,0.035365
